In [ ]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 6.2 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
client_id = '4f8e9219c71d42498f6724a465505cb9'
client_secret = '284f7ae276594d4b80219ba6e022cef2'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '4f8e9219c71d42498f6724a465505cb9'
client_secret = '284f7ae276594d4b80219ba6e022cef2'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

results = spotify.track("16CJ6odC94xBPLaFTSReo1")
print(results['artists'])
for x in results['artists']:
  print(x['id'], x['name'])

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6kFaOeTEkeGZ7oDqJDhC2q'}, 'href': 'https://api.spotify.com/v1/artists/6kFaOeTEkeGZ7oDqJDhC2q', 'id': '6kFaOeTEkeGZ7oDqJDhC2q', 'name': 'La Kultura', 'type': 'artist', 'uri': 'spotify:artist:6kFaOeTEkeGZ7oDqJDhC2q'}]
6kFaOeTEkeGZ7oDqJDhC2q La Kultura


# **Crawl artist**

In [ ]:
# Đọc file album.csv
album_df = pd.read_csv("/content/drive/My Drive/Kì 6/OLAP TNTN+LTD/dataset/spotify-albums_data_2023.csv")

# Tạo các cột artist_id và artist_name nếu chưa có
if 'artist_id' not in album_df.columns:
    album_df['artist_id'] = ''
if 'artist_name' not in album_df.columns:
    album_df['artist_name'] = ''

# Duyệt qua từng track_id và cập nhật thông tin nghệ sĩ
for index, row in album_df.iterrows():
    track_id = row['track_id']
    try:
        results = spotify.track(track_id)
        if results['artists']:
            first_artist = results['artists'][0]
            artist_id = first_artist['id']
            artist_name = first_artist['name']

            # Cập nhật vào DataFrame
            album_df.at[index, 'artist_id'] = artist_id
            album_df.at[index, 'artist_name'] = artist_name
    except Exception as e:
        print(f"Error fetching data for track_id {track_id}: {e}")

# Lưu lại file album.csv
album_df.to_csv('album.csv', index=False)

# Đọc file artist.csv
artist_df = pd.read_csv("/content/drive/My Drive/Kì 6/OLAP TNTN+LTD/dataset/spotify_artist_data_2023.csv")

# Cập nhật thông tin nghệ sĩ trong artist.csv nếu cần
for index, row in artist_df.iterrows():
    artist_id = row['id']
    # Tìm trong album_df để lấy tên nghệ sĩ
    matching_artist = album_df[album_df['artist_id'] == artist_id]
    if not matching_artist.empty:
        artist_name = matching_artist.iloc[0]['artist_name']
        # Cập nhật tên nghệ sĩ
        artist_df.at[index, 'name'] = artist_name

# Lưu lại file artist.csv
artist_df.to_csv('artist.csv', index=False)

# **Tiền xử lý**

In [ ]:
import pandas as pd
import re
from google.colab import drive

# Biểu thức chính quy để tìm các kí tự không mong muốn
pattern = r'[^\x00-\x7F]'

# Hàm kiểm tra xem một chuỗi có chứa kí tự không mong muốn hay không
def contains_unwanted_chars(text):
    if re.search(pattern, str(text)):
        return True
    return False

In [ ]:
def contains_non_english(text):
    # Sử dụng regex để kiểm tra xem chuỗi có ký tự không phải là tiếng Anh không
    return not bool(re.match(r'^[a-zA-Z\s]*$', str(text)))

**Track file**

In [ ]:
# Đọc dữ liệu
df = pd.read_csv("/content/drive/My Drive/Kì 6/OLAP TNTN+LTD/dataset/spotify_tracks_data_2023.csv")

# Giữ lại các dòng có cột "id" là duy nhất
filtered_df = df.drop_duplicates(subset=['id'])

# Xóa rỗng
filtered_df = filtered_df.dropna()

# Lưu DataFrame đã lọc vào file CSV mới
output_path = 'filtered_track.xlsx'
filtered_df.to_excel(output_path, index=False)

**Feature file**

In [ ]:
# Đọc dữ liệu
df = pd.read_csv("/content/drive/My Drive/Kì 6/OLAP TNTN+LTD/dataset/spotify_features_data_2023.csv")

# Giữ lại các dòng có cột "id" là duy nhất
filtered_df = df.drop_duplicates(subset=['id'])

# Xóa rỗng
filtered_df = filtered_df.dropna()

# Lưu DataFrame đã lọc vào file CSV mới
output_path = 'filtered_feature.xlsx'
filtered_df.to_excel(output_path, index=False)

**Artist file**

In [ ]:
# Đọc dữ liệu
df = pd.read_csv("/content/drive/My Drive/Kì 6/OLAP TNTN+LTD/dataset/spotify_artist_data_2023.csv"
, usecols=lambda column: column not in ['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6'])

# Lọc các hàng chứa kí tự không mong muốn
filtered_df = df[~(df['name'].apply(contains_unwanted_chars))]

# Loại bỏ các dòng chứa ký tự không phải ASCII
filtered_df = filtered_df[filtered_df.applymap(lambda x: all(ord(c) < 128 for c in str(x)))]

# Loại bỏ các hàng có giá trị '#NAME?' trong bất kỳ cột nào
filtered_df = filtered_df[(filtered_df != '#NAME?')]

# Xóa dòng có giá trị bắt đầu bằng số hoặc kí tự đặc biệt
pattern = r'^[^a-zA-Z]'  # Kí tự bắt đầu không phải là chữ cái
filtered_df = filtered_df[~filtered_df['name'].astype(str).str.match(pattern)]

# Xóa dòng chứa kí tự $
filtered_df = filtered_df[~filtered_df['name'].astype(str).str.contains('\$')]

# Giữ lại các dòng có cột "id" là duy nhất
filtered_df = filtered_df.drop_duplicates(subset=['id'])

# Xóa rỗng
filtered_df = filtered_df.dropna()

# Lưu DataFrame đã lọc vào file CSV mới
output_path = 'filtered_artist.xlsx'
filtered_df.to_excel(output_path, index=False)

KeyError: 'genre'

**Album file**

In [ ]:
# Đọc dữ liệu
df = pd.read_csv("/content/spotify-albums_data_2023.csv"
, usecols=lambda column: column not in ['artists', 'artist_7', 'artist_8', 'artist_9', 'artist_10', 'artist_11'])

# Lọc các hàng chứa kí tự không mong muốn
filtered_df = df[~(df['track_name'].apply(contains_unwanted_chars)) &
                 ~(df['album_name'].apply(contains_unwanted_chars)) &
                 ~(df['artist_0'].apply(contains_unwanted_chars)) &
                 ~(df['artist_1'].apply(contains_unwanted_chars)) &
                 ~(df['artist_2'].apply(contains_unwanted_chars)) &
                 ~(df['artist_3'].apply(contains_unwanted_chars)) &
                 ~(df['artist_4'].apply(contains_unwanted_chars)) &
                 ~(df['artist_5'].apply(contains_unwanted_chars)) &
                 ~(df['artist_6'].apply(contains_unwanted_chars)) &
                 ~(df['label'].apply(contains_unwanted_chars)) ]

# Loại bỏ các hàng có giá trị '#NAME?' trong bất kỳ cột nào
filtered_df = filtered_df[(filtered_df != '#NAME?')]

# Xóa những dòng có giá trị trong cột 'album_name' bắt đầu bằng số hoặc kí tự đặc biệt không phải là chữ cái
pattern = r'^[^a-zA-Z]'  # Kí tự bắt đầu không phải là chữ cái
filtered_df = filtered_df[~filtered_df['album_name'].astype(str).str.match(pattern)]
filtered_df = filtered_df[~filtered_df['track_name'].astype(str).str.match(pattern)]
filtered_df = filtered_df[~filtered_df['artist_0'].astype(str).str.match(pattern)]

filtered_df = filtered_df[~filtered_df['album_name'].astype(str).str.contains('\$')]
filtered_df = filtered_df[~filtered_df['track_name'].astype(str).str.contains('\$')]
filtered_df = filtered_df[~filtered_df['artist_0'].astype(str).str.contains('\$')]

# Loại bỏ các dòng chứa ký tự không phải ASCII
filtered_df = filtered_df[filtered_df.applymap(lambda x: all(ord(c) < 128 for c in str(x)))]

# Lưu DataFrame đã lọc vào file CSV mới
output_path = 'filtered_album.xlsx'
filtered_df.to_excel(output_path, index=False)

KeyboardInterrupt: 

In [ ]:
df = pd.read_excel("/content/filtered_album.xlsx")

# Xóa rỗng
df = df.dropna(subset=['track_name', 'album_name', 'label', 'artist_0'])

# Các cột cần kiểm tra và làm sạch
columns_to_check = ['track_name', 'album_name', 'label', 'artist_0', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'artist_5', 'artist_6']

# Xác định mẫu regex để tìm các kí tự đặc biệt không phải chữ
pattern = re.compile(r'[^a-zA-Z]')

# Lặp qua từng cột và xóa các dòng có kí tự đặc biệt không phải chữ
for column in columns_to_check:
    df = df[~df[column].astype(str).str.contains(pattern)]

output_path = 'filtered_album1.xlsx'
df.to_excel(output_path, index=False)

In [ ]:
df = pd.read_excel("/content/filtered_album.xlsx")

# Xóa rỗng
df = df.dropna(subset=['track_name', 'album_name', 'release_date', 'label', 'artist_0'])


# Chuyển cột release_date thành kiểu datetime và loại bỏ thông tin về múi giờ
df['release_date'] = pd.to_datetime(df['release_date']).dt.tz_localize(None)

# Lọc ra những dòng có release_date từ năm 1990 trở về trước
df = df[df['release_date'].dt.year > 2000]

# Ghi lại dữ liệu vào tệp Excel mới
df.to_excel('cleaned_album.xlsx', index=False)

**Xóa kí tự đầu**

In [ ]:
df = pd.read_excel("/content/join.xlsx")

# Xóa những dòng có giá trị trong cột 'album_name' bắt đầu bằng số hoặc kí tự đặc biệt không phải là chữ cái
pattern = r'^[^a-zA-Z]'  # Kí tự bắt đầu không phải là chữ cái
df = df[~df['album_name'].astype(str).str.match(pattern)]
df = df[~df['track_name'].astype(str).str.match(pattern)]
df = df[~df['artist_name'].astype(str).str.match(pattern)]

df = df[~df['album_name'].astype(str).str.contains('\$')]
df = df[~df['track_name'].astype(str).str.contains('\$')]
df = df[~df['artist_name'].astype(str).str.contains('\$')]

df = df.dropna()

output_path = 'filtered_join.xlsx'
df.to_excel(output_path, index=False)